In [1]:
import db_utils
import yaml, pandas as pd, numpy as np
from scipy import stats

'''
Loading the data in the yaml file and saving it as 
a csv file on the local machine
'''

with open('credentials.yaml') as file:
    credentials_dict = yaml.safe_load(file)
    
credentials = db_utils.RDSDatabaseConnector(credentials_dict)
loan_payments = credentials.initialise_database()
credentials.save_file(loan_payments, 'new_file.csv')


'''
Loading the data and adjusting the data types of each column of the dataframe
where needed as well as ensuring the formatting of the data (specifically those 
that consist of dates) are all the same
'''

database = db_utils.load_csv('new_file.csv')
cleaned_data = db_utils.DataTransform(database)

date_data = ['issue_date', 'earliest_credit_line', 'last_payment_date', 'next_payment_date',
             'last_credit_pull_date']

categorical_data = ['member_id', 'term', 'int_rate', 'grade', 'sub_grade', 'employment_length', 'home_ownership', 'verification_status', 'loan_status', 
                    'payment_plan', 'purpose', 'policy_code', 'application_type']

non_numeric_data = date_data + categorical_data 
column_headings = database.columns.values.tolist()
numeric_data = [column for column in column_headings if column not in non_numeric_data]

for date_column in date_data:
    database, date_column = cleaned_data.date_data(database, date_column)

for categories in categorical_data:
    database, categories = cleaned_data.change_type(database, categories, 'category')


'''
Dropping columns whose data consists of more than 50% of null values
and filling in the null values for the other columns with null values
'''

df_info = db_utils.DataFrameInfo(database)
df_transform = db_utils.DataFrameTransform(database)
df_plot = db_utils.Plotter(database)

for column in column_headings:
    null_vals, null_percentage = df_info.missing(database, column)
    if null_percentage > float(50):
        database = database.drop(column, axis=1)

database = database.sort_values(by=['sub_grade'])
database['int_rate'] = database['int_rate'].ffill()
database['funded_amount'] = database.loc[:, 'loan_amount']
fill_values = {'collections_12_mths_ex_med': 0, 'term': database['term'].mode()[0]}
database = df_transform.fill_null(database, values= fill_values)

last_payment_date_list= db_utils.make_list(database, 'last_payment_date')
last_credit_pull_date_list = db_utils.make_list(database, 'last_credit_pull_date')

for index in range(54231):
    if pd.isnull(last_payment_date_list[index]):
        last_payment_date_list[index] = last_credit_pull_date_list[index]
    
    if pd.isnull(last_credit_pull_date_list[index]):
        last_credit_pull_date_list[index] = last_payment_date_list[index]

database['last_payment_date'] = last_payment_date_list
database['last_credit_pull_date'] = last_credit_pull_date_list
database = database.dropna(axis = 0)


'''
Finding the skew of the columns made of integers and floats in the
dataframe
'''

df_skew = database.skew(axis= 0, numeric_only= True)
skewed_data = database[[skewed_col for skewed_col, skew_value in df_skew.items() if skew_value > 2]]
#df_plot.plot_hist(skewed_data)

nonskew_data = log_skewed_data = boxcox_skew = yeojohnson_skew = skewed_data.copy()
for column in skewed_data:
    log_skewed_data[column] = log_skewed_data[column].map(lambda i: np.log(i) if i > 0 else 0)

for yeojohnson_col in yeojohnson_skew.columns:
    yeojohnson_skew[yeojohnson_col] = stats.yeojohnson(yeojohnson_skew[yeojohnson_col])[0]

log_skew = log_skewed_data.skew()
yeo_skew = yeojohnson_skew.skew()

print(f'Log: {log_skew} \nYeo Johnson: {yeo_skew}')

for column in skewed_data.columns:
    if abs(log_skew[column]) < abs(yeo_skew[column]):
        nonskew_data[column] = log_skewed_data[column]
    elif abs(yeo_skew[column]) < abs(log_skew[column]):
        nonskew_data[column] = yeojohnson_skew[column]
    else:
        nonskew_data[column] = yeojohnson_skew[column]


for skew_col, value in log_skew.items():
      print(f'{skew_col}
 Skew before: {df_skew.get(key = skew_col)}
 Skew after: {log_skew.get(key = skew_col)}')
for skew_col2, value2 in yeo_skew.items():
      print(f'{skew_col2}
 Skew before: {df_skew.get(key = skew_col2)}
 Skew after: {yeo.get(key = skew_col2)}')
#df_plot.plot_hist(log_skewed_data)

SyntaxError: unterminated string literal (detected at line 110) (1087205676.py, line 110)